In [1]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp

In [2]:
MTL_ConV_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_ConV-Capsnet.xlsx", sheet_name='Sheet1', header=0)
MTL_ConV_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_ConV-FC.xlsx", sheet_name='Sheet1', header=0)
MTL_Deform_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_Deform-Capsnet.xlsx", sheet_name='Sheet1', header=0)
MTL_Deform_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_Deform-FC.xlsx", sheet_name='Sheet1', header=0)

MTL_ConV_Capsnet = MTL_ConV_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_ConV_FC = MTL_ConV_FC.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_Deform_Capsnet = MTL_Deform_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_Deform_FC = MTL_Deform_FC.loc[:,['Dice','F1-Score','Inference_Time', 'Accuracy']]

ST_ConV_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_ConV-Capsnet.xlsx", sheet_name='Sheet1', header=0)
ST_ConV_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_ConV-FC.xlsx", sheet_name='Sheet1', header=0)
ST_Deform_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_Deform-Capsnet.xlsx", sheet_name='Sheet1', header=0)
ST_Deform_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_Deform-FC.xlsx", sheet_name='Sheet1', header=0)

ST_ConV_Capsnet = ST_ConV_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
ST_ConV_FC = ST_ConV_FC.loc[:,['Dice','F1-Score','Inference_Time', 'Accuracy']]
ST_Deform_Capsnet = ST_Deform_Capsnet.loc[:,['Dice','F1-Score','Inference_Time', 'Accuracy']]
ST_Deform_FC = ST_Deform_FC.loc[:,['Dice','F1-Score','Inference_Time', 'Accuracy']]

## **SEGMENTATION**

In [3]:
MTL_Dice = pd.concat([
    MTL_ConV_Capsnet['Dice'],
    MTL_ConV_FC['Dice'],
    MTL_Deform_Capsnet['Dice'],
    MTL_Deform_FC['Dice']
], ignore_index=True)

ST_Dice = pd.concat([
    ST_ConV_Capsnet['Dice'],
    ST_ConV_FC['Dice'],
    ST_Deform_Capsnet['Dice'],
    ST_Deform_FC['Dice']
], ignore_index=True)

# Tạo DataFrame mới
df_dice = pd.DataFrame({
    'MTL_Dice': MTL_Dice,
    'ST_Dice': ST_Dice
})


In [4]:
alpha = 0.05

p_mtl_shapiro = shapiro(MTL_Dice)
p_st_shapiro = shapiro(ST_Dice)

print(f'P-value của kiểm định Shapiro-Wilk cho MTL_Dice: {p_mtl_shapiro.pvalue:.4f}')
print(f'P-value của kiểm định Shapiro-Wilk cho ST_Dice: {p_st_shapiro.pvalue:.4f}')

P-value của kiểm định Shapiro-Wilk cho MTL_Dice: 0.0000
P-value của kiểm định Shapiro-Wilk cho ST_Dice: 0.0000


In [5]:
print(f"--- Thực hiện kiểm định so sánh giữa ST và MTL ---\n")

if p_st_shapiro.pvalue > alpha and p_mtl_shapiro.pvalue > alpha:
    print("Cả hai nhóm đều tuân theo phân phối chuẩn. Sử dụng Independent Samples t-test.")
    stat, p_value = ttest_ind(ST_Dice, MTL_Dice, equal_var=False) # Welch's t-test
    print(f"t-statistic = {stat:.4f}, p-value = {p_value:.4f}")
else:
    print("Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.")
    stat, p_value = mannwhitneyu(ST_Dice, MTL_Dice, alternative='two-sided')
    print(f"U-statistic = {stat:.4f}, p-value = {p_value:.4f}")


--- Thực hiện kiểm định so sánh giữa ST và MTL ---

Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.
U-statistic = 6925.5000, p-value = 0.6104


In [6]:
print("\n--- Kết luận cuối cùng ---")
if p_value < alpha:
    if np.mean(ST_Dice) > np.mean(MTL_Dice):
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất SEGMENTATION của Single-Task tốt hơn Multi-Task.")
    else:
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất SEGMENTATION của Multi-Task tốt hơn Single-Task.")
else:
    print(f"Không có sự khác biệt ý nghĩa thống kê (p >= {alpha}) giữa hiệu suất SEGMENTATION của Single-Task và Multi-Task.")

print(f'Avg_MTL_Dice:\t {np.mean(MTL_Dice):.4f}')
print(f'Avg_ST_Dice:\t {np.mean(ST_Dice):.4f}')


--- Kết luận cuối cùng ---
Không có sự khác biệt ý nghĩa thống kê (p >= 0.05) giữa hiệu suất SEGMENTATION của Single-Task và Multi-Task.
Avg_MTL_Dice:	 0.7134
Avg_ST_Dice:	 0.7053


## **CLASSIFICATION**

In [7]:
MTL_F1 = pd.concat([
    MTL_ConV_Capsnet['F1-Score'],
    MTL_ConV_FC['F1-Score'],
    MTL_Deform_Capsnet['F1-Score'],
    MTL_Deform_FC['F1-Score']
], ignore_index=True)

ST_F1 = pd.concat([
    ST_ConV_Capsnet['F1-Score'],
    ST_ConV_FC['F1-Score'],
    ST_Deform_Capsnet['F1-Score'],
    ST_Deform_FC['F1-Score']
], ignore_index=True)

# Tạo DataFrame mới
df_dice = pd.DataFrame({
    'MTL_F1': MTL_F1,
    'ST_F1': ST_F1
})


In [8]:
alpha = 0.05

p_mtl_shapiro = shapiro(MTL_F1)
p_st_shapiro = shapiro(ST_F1)

print(f'P-value của kiểm định Shapiro-Wilk cho MTL_Dice: {p_mtl_shapiro.pvalue:.4f}')
print(f'P-value của kiểm định Shapiro-Wilk cho ST_Dice: {p_st_shapiro.pvalue:.4f}')

P-value của kiểm định Shapiro-Wilk cho MTL_Dice: 0.0091
P-value của kiểm định Shapiro-Wilk cho ST_Dice: 0.0000


In [9]:
print(f"--- Thực hiện kiểm định so sánh giữa ST và MTL ---\n")

if p_st_shapiro.pvalue > alpha and p_mtl_shapiro.pvalue > alpha:
    print("Cả hai nhóm đều tuân theo phân phối chuẩn. Sử dụng Independent Samples t-test.")
    stat, p_value = ttest_ind(ST_F1, MTL_F1, equal_var=False) # Welch's t-test
    print(f"t-statistic = {stat:.4f}, p-value = {p_value:.4f}")
else:
    print("Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.")
    stat, p_value = mannwhitneyu(ST_F1, MTL_F1, alternative='two-sided')
    print(f"U-statistic = {stat:.4f}, p-value = {p_value:.4f}")


--- Thực hiện kiểm định so sánh giữa ST và MTL ---

Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.
U-statistic = 39.0000, p-value = 0.0000


In [10]:
print("\n--- Kết luận cuối cùng ---")
if p_value < alpha:
    if np.mean(ST_F1) > np.mean(MTL_F1):
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất CLASSIFICATION của Single-Task tốt hơn Multi-Task.")
    else:
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất CLASSIFICATION của Multi-Task tốt hơn Single-Task.")
else:
    print(f"Không có sự khác biệt ý nghĩa thống kê (p >= {alpha}) giữa hiệu suất CLASSIFICATION của Single-Task và Multi-Task.")
    
print(f'Avg_MTL_F1:\t {np.mean(MTL_F1):.4f}')
print(f'Avg_ST_F1:\t {np.mean(ST_F1):.4f}')


--- Kết luận cuối cùng ---
Có sự khác biệt ý nghĩa thống kê (p < 0.05). Hiệu suất CLASSIFICATION của Multi-Task tốt hơn Single-Task.
Avg_MTL_F1:	 0.8529
Avg_ST_F1:	 0.6190


## **Inference Time**

In [11]:
MTL_Time = pd.concat([
    MTL_ConV_Capsnet['Inference_Time'],
    MTL_ConV_FC['Inference_Time'],
    MTL_Deform_Capsnet['Inference_Time'],
    MTL_Deform_FC['Inference_Time']
], ignore_index=True)

ST_Time = pd.concat([
    ST_ConV_Capsnet['Inference_Time'],
    ST_ConV_FC['Inference_Time'],
    ST_Deform_Capsnet['Inference_Time'],
    ST_Deform_FC['Inference_Time']
], ignore_index=True)


alpha = 0.05

p_mtl_shapiro = shapiro(MTL_Time)
p_st_shapiro = shapiro(ST_Time)

print(f'P-value của kiểm định Shapiro-Wilk cho MTL_Dice: {p_mtl_shapiro.pvalue:.4f}')
print(f'P-value của kiểm định Shapiro-Wilk cho ST_Dice: {p_st_shapiro.pvalue:.4f}')

print(f"--- Thực hiện kiểm định so sánh giữa ST và MTL ---\n")

if p_st_shapiro.pvalue > alpha and p_mtl_shapiro.pvalue > alpha:
    print("Cả hai nhóm đều tuân theo phân phối chuẩn. Sử dụng Independent Samples t-test.")
    stat, p_value = ttest_ind(ST_Time, MTL_Time, equal_var=False) # Welch's t-test
    print(f"t-statistic = {stat:.4f}, p-value = {p_value:.4f}")
else:
    print("Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.")
    stat, p_value = mannwhitneyu(ST_Time, MTL_Time, alternative='two-sided')
    print(f"U-statistic = {stat:.4f}, p-value = {p_value:.4f}")

print("\n--- Kết luận cuối cùng ---")
if p_value < alpha:
    if np.mean(ST_Time) < np.mean(MTL_Time):
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Thời gian thực hiện của Single-Task tốt hơn Multi-Task.")
    else:
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Thời gian thực hiện của Multi-Task tốt hơn Single-Task.")
else:
    print(f"Không có sự khác biệt ý nghĩa thống kê (p >= {alpha}) giữa Thời gian thực hiện của Single-Task và Multi-Task.")
    
print(f'Avg_MTL_Time:\t {np.mean(MTL_Time):.4f}')
print(f'Avg_ST_Time:\t {np.mean(ST_Time):.4f}')

P-value của kiểm định Shapiro-Wilk cho MTL_Dice: 0.0000
P-value của kiểm định Shapiro-Wilk cho ST_Dice: 0.0000
--- Thực hiện kiểm định so sánh giữa ST và MTL ---

Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.
U-statistic = 4330.0000, p-value = 0.0000

--- Kết luận cuối cùng ---
Có sự khác biệt ý nghĩa thống kê (p < 0.05). Thời gian thực hiện của Single-Task tốt hơn Multi-Task.
Avg_MTL_Time:	 0.0658
Avg_ST_Time:	 0.0305


## **Accuracy**

In [12]:
MTL_Acc = pd.concat([
    MTL_ConV_Capsnet['Accuracy'],
    MTL_ConV_FC['Accuracy'],
    MTL_Deform_Capsnet['Accuracy'],
    MTL_Deform_FC['Accuracy']
], ignore_index=True)

ST_Acc = pd.concat([
    ST_ConV_Capsnet['Accuracy'],
    ST_ConV_FC['Accuracy'],
    ST_Deform_Capsnet['Accuracy'],
    ST_Deform_FC['Accuracy']
], ignore_index=True)


alpha = 0.05

p_mtl_shapiro = shapiro(MTL_Acc)
p_st_shapiro = shapiro(ST_Acc)

print(f'P-value của kiểm định Shapiro-Wilk cho MTL_Dice: {p_mtl_shapiro.pvalue:.4f}')
print(f'P-value của kiểm định Shapiro-Wilk cho ST_Dice: {p_st_shapiro.pvalue:.4f}')

print(f"--- Thực hiện kiểm định so sánh giữa ST và MTL ---\n")

if p_st_shapiro.pvalue > alpha and p_mtl_shapiro.pvalue > alpha:
    print("Cả hai nhóm đều tuân theo phân phối chuẩn. Sử dụng Independent Samples t-test.")
    stat, p_value = ttest_ind(ST_Acc, MTL_Acc, equal_var=False) # Welch's t-test
    print(f"t-statistic = {stat:.4f}, p-value = {p_value:.4f}")
else:
    print("Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.")
    stat, p_value = mannwhitneyu(ST_Acc, MTL_Acc, alternative='two-sided')
    print(f"U-statistic = {stat:.4f}, p-value = {p_value:.4f}")

print("\n--- Kết luận cuối cùng ---")
if p_value < alpha:
    if np.mean(ST_Acc) > np.mean(MTL_Acc):
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất CLASSIFICATION của Single-Task tốt hơn Multi-Task.")
    else:
        print(f"Có sự khác biệt ý nghĩa thống kê (p < {alpha}). Hiệu suất CLASSIFICATION của Multi-Task tốt hơn Single-Task.")
else:
    print(f"Không có sự khác biệt ý nghĩa thống kê (p >= {alpha}) giữa Thời gian thực hiện của Single-Task và Multi-Task.")
    
print(f'Avg_MTL_Acc:\t {np.mean(MTL_Acc):.4f}')
print(f'Avg_ST_Acc:\t {np.mean(ST_Acc):.4f}')

P-value của kiểm định Shapiro-Wilk cho MTL_Dice: 0.2780
P-value của kiểm định Shapiro-Wilk cho ST_Dice: 0.0002
--- Thực hiện kiểm định so sánh giữa ST và MTL ---

Có ít nhất một nhóm không tuân theo phân phối chuẩn. Sử dụng Mann-Whitney U Test.
U-statistic = 21.0000, p-value = 0.0000

--- Kết luận cuối cùng ---
Có sự khác biệt ý nghĩa thống kê (p < 0.05). Hiệu suất CLASSIFICATION của Multi-Task tốt hơn Single-Task.
Avg_MTL_Acc:	 0.8531
Avg_ST_Acc:	 0.6196
